In [13]:
import numpy as np 
import pandas as pd 

month_yr_store_loc_product_rollup_data = pd.read_csv('clean_data/month_yr_store_loc_product_rollup_data.csv')
store_city_loc_elasticities = pd.read_csv('clean_data/elasticities/store_city_loc_elasticities.csv')
month_yr_store_loc_product_rollup_data.head()

,Year,Month,Product_ID,Product_Name,Product_Category,Product_Cost,Product_Price,Store_City,Store_Location,Units,Actual_Product_Price,Revenue,xRevenue,Profit
0,2022,1,1,Action Figure,Toys,9.99,15.99,Aguascalientes,Downtown,45,15.752889,708.88,719.55,259.33
1,2022,1,1,Action Figure,Toys,9.99,15.99,Campeche,Commercial,20,15.861000,317.22,319.80,117.42
2,2022,1,1,Action Figure,Toys,9.99,15.99,Campeche,Downtown,79,15.326835,1210.82,1263.21,421.61
3,2022,1,1,Action Figure,Toys,9.99,15.99,Chetumal,Downtown,116,15.676810,1818.51,1854.84,659.67
4,2022,1,1,Action Figure,Toys,9.99,15.99,Chihuahua,Commercial,18,15.651667,281.73,287.82,101.91


In [14]:
store_city_loc_elasticities.head(2)

,Store_City,Store_Location,Product_ID,Elasticity
0,Aguascalientes,Downtown,1,-0.082805
1,Aguascalientes,Downtown,2,-0.171935


In [15]:
month_yr_store_product_elasticity_rollup_data = month_yr_store_loc_product_rollup_data.merge(store_city_loc_elasticities,how='left',on=['Store_City','Store_Location','Product_ID'])

checking for null value

In [16]:
month_yr_store_product_elasticity_rollup_data.isna().sum()

Year                    0
Month                   0
Product_ID              0
Product_Name            0
Product_Category        0
Product_Cost            0
Product_Price           0
Store_City              0
Store_Location          0
Units                   0
Actual_Product_Price    0
Revenue                 0
xRevenue                0
Profit                  0
Elasticity              0
dtype: int64

while optimizing the prices, we will consider only last 6 months data. This is because we will have multiple price increases and we would want to check the latest data for prices.

In [17]:
month_yr_store_product_elasticity_rollup_data.loc[month_yr_store_product_elasticity_rollup_data['Year']==2023].Month.max()

np.int64(9)

We will consider only data from 3rd month(March onwards)

In [18]:
last_6_month_yr_store_product_elasticity_rollup_data = month_yr_store_product_elasticity_rollup_data.loc[(month_yr_store_product_elasticity_rollup_data['Year']==2023) & (month_yr_store_product_elasticity_rollup_data['Month']>=3)]

In [19]:
last_6_month_yr_store_product_elasticity_rollup_data = last_6_month_yr_store_product_elasticity_rollup_data.groupby(['Store_City','Store_Location','Product_ID','Product_Name','Product_Category']).agg({'Product_Cost':'max','Units':'sum','Actual_Product_Price':'mean','Product_Price':'max','Revenue':'sum','xRevenue':'sum','Profit':'sum','Elasticity':'mean'}).reset_index()


Constraints
 - Prices to be increased in 0.05 steps (5 cents)
 - Price increase at max will be min(0.1*Current Price,50 cents)
 - Loss of units will be at max 1%
 - No price decreases allowed



In [20]:
def calculate_new_price_and_units(row):
    elasticity = row["Elasticity"]
    current_price = row["Product_Price"]
    units_sold = row["Units"]
    current_revenue = row["xRevenue"]

   # Constraints
    max_price_increase = min(0.50, 0.10 * current_price)  # Maximum price increase
    max_price = current_price + max_price_increase       # Maximum allowed price
    price_steps = np.arange(current_price, max_price+0.04, 0.05)  # Price steps

    best_price = current_price  # Default to current price if no better price is found
    best_units = units_sold     # Default to current units if no better price is found

    # Iterate over potential new prices
    for new_price in price_steps:
        # Calculate new units and revenue
        new_units = units_sold * (1 + elasticity * (new_price - current_price) / current_price)
        new_revenue = new_price * new_units

        # Check constraints: revenue > current_revenue, units loss ≤ 2%
        if new_revenue > current_revenue and (units_sold - new_units) / units_sold <= 0.01:
            best_price = new_price  # Update best price if valid
            best_units = new_units  # Update corresponding units

    return pd.Series([best_price, best_units])

# Apply the function to each row
last_6_month_yr_store_product_elasticity_rollup_data[["New_Product_Price", "New_Units"]] = last_6_month_yr_store_product_elasticity_rollup_data.apply(calculate_new_price_and_units, axis=1)
last_6_month_yr_store_product_elasticity_rollup_data['New_Product_Price'] = (
    last_6_month_yr_store_product_elasticity_rollup_data["New_Product_Price"]
    .apply(lambda x: x + 0.01 if str(round(x, 2))[-1] == '4' else x)
)
last_6_month_yr_store_product_elasticity_rollup_data['New_Revenue'] = round(last_6_month_yr_store_product_elasticity_rollup_data["New_Product_Price"]*last_6_month_yr_store_product_elasticity_rollup_data["New_Units"],2)
last_6_month_yr_store_product_elasticity_rollup_data['Delta_Revenue'] = last_6_month_yr_store_product_elasticity_rollup_data['New_Revenue'] -last_6_month_yr_store_product_elasticity_rollup_data['xRevenue']
last_6_month_yr_store_product_elasticity_rollup_data.head()

,Store_City,Store_Location,Product_ID,Product_Name,Product_Category,Product_Cost,Units,Actual_Product_Price,Product_Price,Revenue,xRevenue,Profit,Elasticity,New_Product_Price,New_Units,New_Revenue,Delta_Revenue
0,Aguascalientes,Downtown,1,Action Figure,Toys,10.99,102,17.178638,17.99,1737.82,1793.98,616.84,-0.082805,18.49,101.765256,1881.64,87.66
1,Aguascalientes,Downtown,2,Animal Figures,Toys,10.49,295,13.317577,13.99,3931.81,4063.55,837.26,-0.171935,14.49,293.187248,4248.28,184.73
2,Aguascalientes,Downtown,3,Barrel O' Slime,Art & Crafts,1.99,792,4.762504,4.99,3685.75,3818.58,2109.67,-0.139763,5.35,784.236014,4195.66,377.08
3,Aguascalientes,Downtown,4,Chutes & Ladders,Games,10.09,41,12.900000,13.49,533.84,553.09,123.75,-0.001000,13.99,40.998480,573.57,20.48
4,Aguascalientes,Downtown,5,Classic Dominoes,Games,7.99,21,10.480000,10.49,220.08,220.29,52.29,-0.001000,10.99,20.998999,230.78,10.49


In [21]:
last_6_month_yr_store_product_elasticity_rollup_data.to_csv('clean_data/region_level_price_optimized_data.csv',index=False)

In [52]:
set(last_6_month_yr_store_product_elasticity_rollup_data.Store_City.to_list())

{'Aguascalientes',
 'Campeche',
 'Chetumal',
 'Chihuahua',
 'Chilpancingo',
 'Ciudad Victoria',
 'Cuernavaca',
 'Cuidad de Mexico',
 'Culiacan',
 'Durango',
 'Guadalajara',
 'Guanajuato',
 'Hermosillo',
 'La Paz',
 'Merida',
 'Mexicali',
 'Monterrey',
 'Morelia',
 'Oaxaca',
 'Pachuca',
 'Puebla',
 'Saltillo',
 'San Luis Potosi',
 'Santiago',
 'Toluca',
 'Tuxtla Gutierrez',
 'Villahermosa',
 'Xalapa',
 'Zacatecas'}